Progetto miscentering:
Per prima cosa dovrete fare un cross-match tra il catalogo di ammassi trovati dall'algoritmo redmapper nella survey DES con i cataloghi di ammassi trovati dalle surveys SPT. Il matching lo potete fare sulle coordinate RA-DEC e sul redshift, diciamo chiedendo che la distanza proiettata sia minore di 10 arcmin e che la differenza tra i redshift sia minore di 0.1. Potete usare le funzioni di astropy sia per leggere i cataloghi in formato fits ( from astropy.table import Table  ... Table.read(..)) sia per fare il matching usando le funzioni di astropy coordinates. Il catalogo redmapper non è ancora pubblico (y3_gold***), quindi vi pregherei di non circolarlo.. I cataloghi SPT li potete scaricare da qui:
https://pole.uchicago.edu/public/data/sptsz-clusters/sptecs_catalog_oct919.fits
https://pole.uchicago.edu/public/data/sptsz-clusters/2500d_cluster_sample_Bocquet19.fits
Le coordinate SPT sono appunto 'RA' 'DEC' e 'REDSHIFT', le coordinate RM sono 'RA', 'DEC' ed il redshift è 'Z_LAMBDA'
Una volta fatto il matching avrete cataloghi identificati sia nell'ottico da RM, sia nel millimetrico da SPT. A questo punto, dato il redshift dell'ammasso assumete una cosmologia standard e convertite la distanza tra la posizione di RM e quella di SPT da distanza proiettata (in arcmin) a distanza fisica in Mpc. Poi avrete quindi una lista di distanze e potete scrivere una likelihood per esempio fittando una distribuzione di rayleigh ai tuoi punti e vincolare sigma. Se siete svelto, potete provare a fittare anche la somma di due distribuzioni di rayleigh assumendo che la popolazione di ammassi sia descritta da una frazione di ammassi con i centri ben identificati e da un'altra popolazione con i centri errati (ma questo punto forse ti richiederà troppo tempo, se intendete farlo fatemi sapere che ti do ulteriori dettagli)

In [1]:
from astropy.table import Table, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord

Visualizing data: 

In [2]:
SPT1 = 'sptecs_catalog_oct919.fitS'
SPT2 = '2500d_cluster_sample_Bocquet19.fits'
RM = 'y3_gold_2.2.1_wide_sofcol_run2_redmapper_v6.4.22+2_lgt20_vl50_catalog.fit'

print(Table.read(SPT1))
print(Table.read(SPT2))
print(Table.read(RM))

     SPT_ID           RA     ...        PHOTOMETRIC_ZREF      
---------------- ----------- ... -----------------------------
SPT-CLJ0000-2518 0.072338357 ...                              
SPT-CLJ0005-3751   1.4383179 ...                              
SPT-CLJ0005-3443   1.4876514 ...                              
SPT-CLJ0012-3537   3.0433397 ...                              
SPT-CLJ0012-3346   3.1111835 ...                              
SPT-CLJ0014-3022   3.5726598 ...                              
SPT-CLJ0014-2024   3.6931852 ...                              
SPT-CLJ0020-2543   5.1730618 ...                              
SPT-CLJ0028-2649   7.0110953 ...                              
SPT-CLJ0035-2015   8.8608249 ...                              
             ...         ... ...                           ...
SPT-CLJ2236-2458    339.0403 ...                              
SPT-CLJ2238-2452   339.62654 ...                              
SPT-CLJ2238-2458   339.68902 ...                       

Now we extract coordinates

In [62]:
ra_SPT1 = Table.read(SPT1)['RA']
ra_SPT2 = Table.read(SPT2)['RA']
dec_SPT1 = Table.read(SPT1)['DEC']
dec_SPT2 = Table.read(SPT2)['DEC']
z_SPT1 = Table.read(SPT1)['REDSHIFT']
z_SPT2 = Table.read(SPT2)['REDSHIFT']
ra_SPT = vstack([ra_SPT1, ra_SPT2])
dec_SPT = vstack([dec_SPT1, dec_SPT2])
z_SPT = vstack([z_SPT1, z_SPT2])

ra_RM = Table.read(RM)['RA']
dec_RM = Table.read(RM)['DEC']
z_RM = Table.read(RM)['Z_LAMBDA']

Let's check dimension and range

In [57]:
print("SPT right ascension goes from", min(ra_SPT['RA']), " degrees to", max(ra_SPT['RA']), "degrees, dimension: ", len(ra_SPT))
print("SPT declination goes from", min(dec_SPT['DEC']), " degrees to", max(dec_SPT['DEC']), "degrees, dimension: ", len(dec_SPT))
print("SPT redshift ascension goes from", min(z_SPT['REDSHIFT']), " to", max(z_SPT['REDSHIFT']), ", dimension: ", len(z_SPT))

print("RM right ascension goes from", min(ra_RM), " degrees to", max(ra_RM), "degrees, dimension: ", len(ra_RM))
print("RM declination goes from", min(dec_RM), " degrees to", max(dec_RM), "degrees, dimension: ", len(dec_RM))
print("RM redshift ascension goes from", min(z_RM), " to", max(z_RM), ", dimension: ", len(z_RM))

SPT right ascension goes from 0.011752804  degrees to 359.92296 degrees, dimension:  1147
SPT declination goes from -65.310319  degrees to -19.838489 degrees, dimension:  1147
SPT redshift ascension goes from 0.0  to 1.7 , dimension:  1147
RM right ascension goes from 0.00427282777269289  degrees to 359.99694655927 degrees, dimension:  53610
RM declination goes from -68.0766988956839  degrees to 5.75057890936839 degrees, dimension:  53610
RM redshift ascension goes from 0.1001067  to 0.9499982 , dimension:  53610


In [65]:
ra_SPT = np.array(ra_SPT['RA'])
dec_SPT = np.array(dec_SPT['DEC'])
type(ra_SPT)

numpy.ndarray

In [66]:
c_SPT = SkyCoord(ra=ra_SPT*u.degree, dec=dec_SPT*u.degree, redshift=z_SPT)
#c_RM = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, distance=distance2*u.kpc)
#>>> idx, d2d, d3d = c.match_to_catalog_3d(catalog)

ValueError: Unrecognized keyword argument(s) 'redshift'